# Compressing and Cleaning Datasets

## Song data from Genius

Compressed the genius song data into only english songs, a range from 1960 to 2019, and views greater than or equal to 5,000 and cleaned the lyrical column

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn

In [ ]:
import pandas as pd

cols = ['title', 'tag', 'artist', 'year', 'views','lyrics', 'language_cld3']


songs_df = pd.read_csv('../data/raw/song_lyrics.csv', usecols=cols, chunksize=100000)

filtered = []

for data in songs_df:
    data = data[(data['language_cld3'] == 'en') & ((data['year'] >= 1960) & (data['year'] <= 2019)) & (data['views'] >= 5000)]

    data['lyrics'] = data['lyrics'].fillna('').astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    filtered.append(data)

df = pd.concat(filtered, ignore_index=True)

#songs_compressed = df.groupby('year').apply(lambda x: x.sample(n=min(len(x), 275), random_state=42, replace=False)).reset_index(drop=True)

df.to_csv('../data/raw/songs_compressed.csv', index=False)

## Merge Genius and Spotify Data & Clean Data

In [4]:
import pandas as pd
import re


def cleaned_title(title):
    t = title.lower().strip()
    t = re.sub(r"\((ft|feat|featuring)[^)]+\)","",t) # remove (feat. Artist)
    t = re.sub(r"(ft|feat|featuring)\s+[^-()]+", "", t) #remove feat. (without parantheses)
    t = re.sub(r"-\s*(edit|remastered|remaster|version|live).*","",t)
    t = " ".join(t.split()) # removing extra spaces
    return t

def cleaned_artist(artist):
    a = artist.lower().strip()
    a = " ".join(a.split()) # removing extra spaces

    return a


lyrics_df = pd.read_csv('../data/raw/songs_compressed.csv')
spotify_df = pd.read_csv('../data/raw/spotify.csv')


spotify_df['artist_list'] = spotify_df['artists'].apply(lambda x: [a.strip().strip("'\"") for a in x[1:-1].split(',')] if isinstance(x, str) else [x]) # converts the string representation of list to actual list of artist names
spotify_df = spotify_df.explode('artist_list') # makes sure that each artists in the list has its own row, so we can match them individually

spotify_df = spotify_df.rename(columns={'name': 'title', 'artist_list': 'artist'})


spotify_df['artist'] = spotify_df['artist'].astype(str)
spotify_df['title'] = spotify_df['title'].astype(str)
lyrics_df['artist'] = lyrics_df['artist'].astype(str)
lyrics_df['title'] = lyrics_df['title'].astype(str)

spotify_df['cleaned_artist'] = spotify_df['artist'].apply(cleaned_artist)
spotify_df['cleaned_title'] = spotify_df['title'].apply(cleaned_title)

lyrics_df['cleaned_artist'] = lyrics_df['artist'].apply(cleaned_artist)
lyrics_df['cleaned_title'] = lyrics_df['title'].apply(cleaned_title)    


merged_df = pd.merge(lyrics_df,spotify_df,on=['cleaned_title', 'cleaned_artist'],how='inner')


merged_df = merged_df.drop(columns=['cleaned_artist', 'cleaned_title', 'views', 'language_cld3', 'key', 'mode', 'time_signature', 'Unnamed: 0', 'id', 'artists_upd_v1',  'artists_upd_v2', 'artists_upd', 'artists_song', 'title_y', 'artist_y', 'popularity', 'duration_ms', 'explicit', 'artists', 'id_artists', 'release_date'])

merged_df = merged_df.rename(columns={'title_x': 'title','artist_x': 'artist','consolidates_genre_lists': 'spotify_genre_list','tag': 'genius_genre'})

merged_df = merged_df.drop_duplicates(subset=['title', 'artist'], keep='first')

merged_df.to_csv('../data/compressed/lyrics_spotify_features.csv', index=False)

print(merged_df.shape)
merged_df.head()

(27620, 15)


,title,genius_genre,artist,year,lyrics,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,spotify_genre_list
0,Can I Live,rap,JAY-Z,1996,"[Produced by Irv Gotti] [Intro] Yeah, hah, yea...",0.628,0.692,-12.365,0.437,0.0823,0.0000,0.161,0.575,76.440,"['pop_rap', 'rap', 'east_coast_hip_hop', 'hip_..."
1,Money On My Mind,rap,Lil Wayne,2005,"[Intro] Yeah Money on my mind, money on my min...",0.535,0.772,-6.503,0.370,0.0127,0.0000,0.110,0.661,152.173,"['trap', 'rap', 'pop_rap', 'hip_hop', 'new_orl..."
2,Mr. Carter,rap,Lil Wayne,2008,[Produced by Infamous and Drew Correa] [Intro:...,0.485,0.710,-6.288,0.364,0.0444,0.0000,0.350,0.473,170.942,"['trap', 'east_coast_hip_hop', 'rap', 'pop_rap..."
3,C.R.E.A.M.,rap,Wu-Tang Clan,1994,[Produced by RZA] [Intro: Raekwon & Method Man...,0.479,0.549,-10.551,0.373,0.5700,0.0239,0.127,0.576,180.985,"['east_coast_hip_hop', 'gangster_rap', 'hardco..."
4,Barry Bonds,rap,Kanye West,2007,[Verse 1: Kanye West] It's what you all been w...,0.480,0.624,-6.131,0.382,0.0451,0.0000,0.337,0.704,165.057,"['chicago_rap', 'trap', 'rap', 'pop_rap', 'hip..."


## BillBoard Data

In [6]:
bbh100_columns_to_keep = ["Artist(s) Genres", "Hot100 Ranking Year", "Hot100 Rank"]
billboard_df = pd.read_csv('../data/raw/billboard.csv', usecols=bbh100_columns_to_keep)

# the year filter
billboard_df = billboard_df[billboard_df["Hot100 Ranking Year"] >= 1960]

billboard_df.to_csv('../data/compressed/new_billboard_compressed.csv', index=False)

In [7]:
billboard_df.head(10)

,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank
586,"['easy listening', ""man's orchestra""]",1960,1
587,['nashville sound'],1960,2
588,"['folk rock', 'rock-and-roll', 'adult standard...",1960,3
589,"['rockabilly', 'doo-wop', 'rock-and-roll']",1960,4
590,['deep adult standards'],1960,5
591,['karaoke'],1960,6
592,"['rockabilly', 'rock-and-roll']",1960,7
593,"['doo-wop', 'rhythm and blues']",1960,8
594,"['rockabilly', 'rock-and-roll']",1960,9
595,"['rockabilly', 'rock-and-roll']",1960,10
